In [1]:
!pip install pyngrok
!pip install --upgrade pyngrok

In [2]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2024-01-28 15:28:04.146897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 15:28:04.146979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Att

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
import string

In [5]:
from pyngrok import ngrok

In [4]:
port_no = 3000

In [ ]:
!ngrok authtoken '2bSStTNdeNyymLGEG3xtHXM0vim_73UeZsM2THq9UfJUjyDz5'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
ngrok.set_auth_token("2bSStTNdeNyymLGEG3xtHXM0vim_73UeZsM2THq9UfJUjyDz5")
public_url =  ngrok.connect(port_no).public_url

In [7]:
print(public_url)

https://91d4-34-141-128-24.ngrok-free.app


In [8]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/api/data', methods=['POST'])
def handle_post_request():
    json_data = request.json
    text = json_data.replace('{', '').replace('}', '').replace("'", '').replace(',', ' , ')
    print('Received JSON data:', text)
    # text = """
    # Diagnosis Category: Infectious, Treatment Category: Pharmacy/Prescription Drugs, Determination: Overturned Decision of Health Plan, Findings: An enrollee has requested Harvoni for treatment of his hepatitis C virus genotype.\n
    # Diagnosis Category: Mental, Treatment Category: Mental Health Treatment, Determination: Upheld Decision of Health Plan, Findings: An enrollee has requested  residential treatment center (RTC) services for treatment of her medical condition.\n
    # Diagnosis Category: Prevention/Good Health, Treatment Category: Diagnostic Imaging, Screening and Testing, Determination: Overturned Decision of Health Plan, Findings: An enrollee has requested breast tomosynthesis for evaluation of her medical condition.
    # """
    stopwords = list(STOP_WORDS)
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    tokens = [token.text for token in doc]

    punctuation = string.punctuation + '\n'

    word_frequencies = {}
    for word in doc:
      if word.text.lower() not in stopwords:
          if word.text.lower() not in punctuation:
              if word.text not in word_frequencies.keys():
                  word_frequencies[word.text] = 1
              else:
                  word_frequencies[word.text] += 1

    max_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
      word_frequencies[word] = word_frequencies[word]/max_frequency

    sentence_tokens = [sent for sent in doc.sents]

    sentence_scores = {}
    for sent in sentence_tokens:
      for word in sent:
          if word.text.lower() in word_frequencies.keys():
              if sent not in sentence_scores.keys():
                  sentence_scores[sent] = word_frequencies[word.text.lower()]
              else:
                  sentence_scores[sent] += word_frequencies[word.text.lower()]

    select_length = int(len(sentence_tokens)*0.6)
    summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
    final_summary = [word.text for word in summary]
    summary = ' '.join(final_summary)

    print(summary)
    response_data = {'status': 'success', 'summary': summary}

    return jsonify(response_data), 200

if __name__ == '__main__':
    app.run(port=port_no)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:3000
INFO:werkzeug:Press CTRL+C to quit


Received JSON data: {'input_text': '{patient_name:zeba musarrat,symptoms:fever,diagnosis:normal fever ,subdiagnosis:n/a,treatment:test,subtreatment:test,determination:test,type:test,findings:test,comments:test},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammation, Muscle Stiffness,diagnosis:Orthopedics,subdiagnosis:Muscle Spasms,treatment:Corticosteroid Injections,subtreatment:Injections,determination:n/a,type:Investigational,findings:n/a,comments:Take rest and prevent from lifting weights},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammation, Muscle Stiffness,diagnosis:Orthopedics,subdiagnosis:Muscle Spasms,treatment:Injection,subtreatment:Corticosteroid Injections,determination:\\n/a,type:Investigational,findings:\\n/a,comments:Take rest and prevent lifting any weights.},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammation, Muscle Stiffness,diagnosis:Orthopedics,subdiagnosis:Muscle Spasms,treatment:Injection,subtreatment:Corticosteroid Injections,determinat

INFO:werkzeug:127.0.0.1 - - [28/Jan/2024 15:39:11] "POST /api/data HTTP/1.1" 200 -


Mental Health Treatment, Determination: Upheld Decision of Health Plan, Findings: An enrollee has requested  residential treatment center (RTC) services for treatment of her medical condition.

     
    Diagnosis Category: Infectious, Treatment Category: Pharmacy/Prescription Drugs, Determination: Overturned Decision of Health Plan, Findings: An enrollee has requested Harvoni for treatment of his hepatitis C virus genotype.

    
Received JSON data: {'input_text': '{patient_name:zeba musarrat,symptoms:fever,diagnosis:normal fever ,subdiagnosis:n/a,treatment:test,subtreatment:test,determination:test,type:test,findings:test,comments:test},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammation, Muscle Stiffness,diagnosis:Orthopedics,subdiagnosis:Muscle Spasms,treatment:Corticosteroid Injections,subtreatment:Injections,determination:n/a,type:Investigational,findings:n/a,comments:Take rest and prevent from lifting weights},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammatio

INFO:werkzeug:127.0.0.1 - - [28/Jan/2024 15:39:30] "POST /api/data HTTP/1.1" 200 -


Mental Health Treatment, Determination: Upheld Decision of Health Plan, Findings: An enrollee has requested  residential treatment center (RTC) services for treatment of her medical condition.

     
    Diagnosis Category: Infectious, Treatment Category: Pharmacy/Prescription Drugs, Determination: Overturned Decision of Health Plan, Findings: An enrollee has requested Harvoni for treatment of his hepatitis C virus genotype.

    
Received JSON data: {'input_text': '{patient_name:zeba musarrat,symptoms:fever,diagnosis:normal fever ,subdiagnosis:n/a,treatment:test,subtreatment:test,determination:test,type:test,findings:test,comments:test},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammation, Muscle Stiffness,diagnosis:Orthopedics,subdiagnosis:Muscle Spasms,treatment:Corticosteroid Injections,subtreatment:Injections,determination:n/a,type:Investigational,findings:n/a,comments:Take rest and prevent from lifting weights},{patient_name:zeba musarrat,symptoms:Joint Pain, Inflammatio

INFO:werkzeug:127.0.0.1 - - [28/Jan/2024 15:44:39] "POST /api/data HTTP/1.1" 200 -


Mental Health Treatment, Determination: Upheld Decision of Health Plan, Findings: An enrollee has requested  residential treatment center (RTC) services for treatment of her medical condition.

     
    Diagnosis Category: Infectious, Treatment Category: Pharmacy/Prescription Drugs, Determination: Overturned Decision of Health Plan, Findings: An enrollee has requested Harvoni for treatment of his hepatitis C virus genotype.

    
